#GA360 Segmentology
GA360 funnel analysis using Census data.


#License

Copyright 2020 Google LLC,

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

  https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.



#Disclaimer
This is not an officially supported Google product. It is a reference implementation. There is absolutely NO WARRANTY provided for using this code. The code is Apache Licensed and CAN BE fully modified, white labeled, and disassembled by your team.

This code generated (see starthinker/scripts for possible source):
  - **Command**: "python starthinker_ui/manage.py colab"
  - **Command**: "python starthinker/tools/colab.py [JSON RECIPE]"



#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Set Configuration

This code is required to initialize the project. Fill in required fields and press play.

1. If the recipe uses a Google Cloud Project:
  - Set the configuration **project** value to the project identifier from [these instructions](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md).

1. If the recipe has **auth** set to **user**:
  - If you have user credentials:
    - Set the configuration **user** value to your user credentials JSON.
  - If you DO NOT have user credentials:
    - Set the configuration **client** value to [downloaded client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md).

1. If the recipe has **auth** set to **service**:
  - Set the configuration **service** value to [downloaded service credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_service.md).



In [ ]:
from starthinker.util.configuration import Configuration


CONFIG = Configuration(
  project="",
  client={},
  service={},
  user="/content/user.json",
  verbose=True
)



#3. Enter GA360 Segmentology Recipe Parameters
 1. Wait for **BigQuery->->->Census_Join** to be created.
 1. Join the [](https://groups.google.com/d/forum/starthinker-assets) to access the following assets
 1. Copy [](https://datastudio.google.com/c/u/0/reporting/3673497b-f36f-4448-8fb9-3e05ea51842f/). Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to **BigQuery->->->Census_Join**.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_write':'service',  # Authorization used for writing data.
  'auth_read':'service',  # Authorization for reading GA360.
  'view':'service',  # View Id
  'recipe_slug':'',  # Name of Google BigQuery dataset to create.
}

print("Parameters Set To: %s" % FIELDS)


#4. Execute GA360 Segmentology
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.configuration import execute
from starthinker.util.recipe import json_set_fields

TASKS = [
  {
    'dataset':{
      'description':'Create a dataset for bigquery tables.',
      'hour':[
        4
      ],
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing data.'}},
      'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'bigquery':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Credentials used for writing function.'}},
      'function':'Pearson Significance Test',
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}}
      }
    }
  },
  {
    'ga':{
      'auth':{'field':{'name':'auth_read','kind':'authentication','order':1,'default':'service','description':'Authorization for reading GA360.'}},
      'kwargs':{
        'reportRequests':[
          {
            'viewId':{'field':{'name':'view','kind':'string','order':2,'default':'service','description':'View Id'}},
            'dateRanges':[
              {
                'startDate':'90daysAgo',
                'endDate':'today'
              }
            ],
            'dimensions':[
              {
                'name':'ga:userType'
              },
              {
                'name':'ga:userDefinedValue'
              },
              {
                'name':'ga:latitude'
              },
              {
                'name':'ga:longitude'
              }
            ],
            'metrics':[
              {
                'expression':'ga:users'
              },
              {
                'expression':'ga:sessionsPerUser'
              },
              {
                'expression':'ga:bounces'
              },
              {
                'expression':'ga:timeOnPage'
              },
              {
                'expression':'ga:pageviews'
              }
            ]
          }
        ],
        'useResourceQuotas':False
      },
      'out':{
        'bigquery':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
          'table':'GA360_KPI'
        }
      }
    }
  },
  {
    'bigquery':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Authorization used for writing data.'}},
      'from':{
        'query':'WITH GA360_SUM AS (           SELECT              A.Dimensions.userType AS User_Type,             A.Dimensions.userDefinedValue AS User_Value,             B.zip_code AS Zip,             SUM(Metrics.users) AS Users,             SUM(Metrics.sessionsPerUser) AS Sessions,             SUM(Metrics.timeOnPage) AS Time_On_Site,             SUM(Metrics.bounces) AS Bounces,             SUM(Metrics.pageviews) AS Page_Views           FROM `{dataset}.GA360_KPI` AS A            JOIN `bigquery-public-data.geo_us_boundaries.zip_codes` AS B           ON ST_WITHIN(ST_GEOGPOINT(A.Dimensions.longitude, A.Dimensions.latitude), B.zip_code_geom)           GROUP BY 1,2,3           )           SELECT             User_Type,             User_Value,             Zip,             Users,             SAFE_DIVIDE(Users, SUM(Users) OVER()) AS User_Percent,             SAFE_DIVIDE(Sessions, SUM(Sessions) OVER()) AS Impression_Percent,             SAFE_DIVIDE(Time_On_Site, SUM(Time_On_Site) OVER()) AS Time_On_Site_Percent,             SAFE_DIVIDE(Bounces, SUM(Bounces) OVER()) AS Bounce_Percent,             SAFE_DIVIDE(Page_Views, SUM(Page_Views) OVER()) AS Page_View_Percent           FROM GA360_SUM        ',
        'parameters':{
          'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be created in BigQuery.'}}
        },
        'legacy':False
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','description':'Place where tables will be written in BigQuery.'}},
        'view':'GA360_KPI_Normalized'
      }
    }
  },
  {
    'census':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Authorization used for writing data.'}},
      'normalize':{
        'census_geography':'zip_codes',
        'census_year':'2018',
        'census_span':'5yr'
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'type':'view'
      }
    }
  },
  {
    'census':{
      'auth':{'field':{'name':'auth_write','kind':'authentication','order':1,'default':'service','description':'Authorization used for writing data.'}},
      'correlate':{
        'join':'Zip',
        'pass':[
          'User_Type',
          'User_Value'
        ],
        'sum':[
          'Users'
        ],
        'correlate':[
          'User_Percent',
          'Impression_Percent',
          'Time_On_Site_Percent',
          'Bounce_Percent',
          'Page_View_Percent'
        ],
        'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'table':'GA360_KPI_Normalized',
        'significance':80
      },
      'to':{
        'dataset':{'field':{'name':'recipe_slug','kind':'string','order':4,'default':'','description':'Name of Google BigQuery dataset to create.'}},
        'type':'view'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

execute(CONFIG, TASKS, force=True)
